In [ ]:
# Author:songxindong
# Created: 2025-10-15
# Last Modified: 2025-10-15 14:27:33
# 技术文档: https://km.sankuai.com/collabpage/2727150323

In [ ]:
# # 定义变量dt,以便做调度策略，真正的取值是t-1
dt = "19960224"

In [1]:
import os
os.environ['JUPYTER_CONDA_ARCHIVE'] = 'viewfs:///user/hadoop-caterb2b-mall-risk-management/jupyter/4f30bed3-2cae-400e-b76e-3339664503d3/archive/codelab-8e4ce7edc4a6eb36eb5ec2d7b13d2276.zip'
os.environ['PYSPARK_DRIVER_PYTHON'] = '//conda/envs/codelab/bin/python'
os.environ['PYSPARK_PYTHON'] = './ARCHIVE/conda/envs/codelab/bin/python'
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--conf spark.executor.cores=2 '
    '--conf spark.kubernetes.memoryOverheadFactor=0.1 '
    '--conf spark.executor.memory=4g '
    'pyspark-shell'
)

In [2]:
%spark_version 3.x

In [3]:
%%spark 
--conf spark.yarn.queue=root.zw05_training_cluster.hadoop-caterb2b.training
--conf spark.yarn.dist.archives=viewfs:///user/hadoop-caterb2b-mall-risk-management/jupyter/4f30bed3-2cae-400e-b76e-3339664503d3/archive/codelab-8e4ce7edc4a6eb36eb5ec2d7b13d2276.zip#ARCHIVE
--conf spark.pyspark.python=./ARCHIVE/conda/envs/codelab/bin/python
--conf spark.pyspark.driver.python=./ARCHIVE/conda/envs/codelab/bin/python
--conf spark.yarn.appMasterEnv.PYSPARK_PYTHON=./ARCHIVE/conda/envs/codelab/bin/python

,Driver日志,队列信息
,/home/sankuai/logs/spark-1760495232.log,root.zw05_training_cluster.hadoop-caterb2b.training


CmdOutput(['tail', '-50', '/home/sankuai/logs/spark-1760495232.log'], 2)

Setting spark.hadoop.yarn.resourcemanager.principal to hadoop-caterb2b-mall-risk-management
SLA info url: http://cantor.data.sankuai.com/api/v1/task/normal_spark_songxindong_8993028c/level
SLA info result: {
  "data": {
    "level": 0, 
    "taskWeight": ""
  }, 
  "success": true
}

Set spark.shuffle.manager=rss because spark.k8s.app.tags.tasktype is ml.
Set spark.rss.hdfs.nameSpace to value : hdfs://zw06-rss-mlprod04/zw06rssmlprod04nn
spark.rss.hdfs.base.nn.index : 3
Set spark.sql.adaptive.localShuffleReader.enabled=false, because rss can not read from local and it may also result read amplification
getExtraConf info url: http://themis.data.sankuai.com/api/spark/whitelist/config/etl/normal_spark_songxindong_8993028c
getExtraConf info result: {
  "success" : false,
  "appName" : "normal_spark_songxindong_8993028c",
  "arguments" : null,
  "enabled" : false
}
set spark.reducer.maxReqsInFlight to value: 10
set spark.reducer.maxBlocksInFlightPerAddress to value : 1
Invalid cantor schedul

,SparkSession,SparkContext,Job Search/applicationId
链接/变量名,spark,sc,spark-application-1760495258290


In [ ]:
# 2. 注册 UDF：JSON → 对话文本
import json  # 导入json模块，用于处理JSON数据
from pyspark.sql import functions as F  # 导入PySpark SQL函数模块，用于创建UDF
from pyspark.sql.types import ArrayType, StringType  # 导入PySpark SQL类型模块，用于定义数据类型

def process_sentence(sentence: str) -> str:
    """
    处理包含对话信息的JSON字符串，将其转换为对话文本的JSON数组字符串。

    参数:
        sentence (str): 输入的JSON字符串，应包含对话信息。

    返回:
        str: 转换后的对话文本JSON数组字符串，若处理失败则返回空字符串。
    """
    try:
        data = json.loads(sentence)  # 解析输入的JSON字符串为Python对象
        speaker_map = {0: "bd", 1: "客户"}  # 定义说话者映射，将频道号映射为角色名称
        conv = []  # 初始化对话列表，用于存储格式化后的对话内容
        if isinstance(data, dict) and "dialog" in data:  # 检查数据是否为字典且包含"dialog"键
            for item in data["dialog"]:  # 遍历对话列表中的每个条目
                speaker = speaker_map.get(item["channel"], f"未知({item['channel']})")  # 获取说话者名称，若未知则使用频道号
                conv.append(f"{speaker}: {item['content']}")  # 格式化对话内容并添加到列表
        return json.dumps(conv, ensure_ascii=False)  # 将对话列表转换为JSON字符串并返回，确保非ASCII字符正确显示
    except Exception:
        return ''  # 若处理过程中发生异常，返回空字符串

process_sentence_udf = F.udf(process_sentence, StringType())  # 注册UDF，将Python函数转换为Spark SQL用户定义函数，返回类型为字符串


In [53]:
# 3. 读原始数据 + 预处理
sdf_raw = spark.sql(f"""
SELECT DISTINCT 
    t3.customer_id,
    t5.customer_name,
    t1.contact_id,
    t3.bd_id,
    t3.bd_name,
    t4.area_name,
    t1.internet_call_type,
    t1.talking_start_time,
    t1.end_time,
    t1.talking_time_len,
    t1.bd_tel_num,
    t1.customer_tel_num,
    t1.record_url,
    t2.voice_info
FROM mart_caterb2b.fact_tel_visit_profile_day t1
INNER JOIN mart_caterb2b.fact_sale_voice_text_day t2 
    ON t1.contact_id = t2.original_contact_id AND t1.dt = t2.dt
INNER JOIN mart_caterb2b.fact_caterb2b_poi_visit t3 
    ON t2.visit_id = t3.id AND t2.dt = t3.dt
LEFT JOIN mart_caterb2b.dim_sm_rac_org_bd_info_his t4 
    ON t3.bd_id = t4.bd_id AND t4.dt = t2.dt
LEFT JOIN mart_caterb2b.dim_caterb2b_customer_his t5
    ON t3.customer_id = t5.customer_id AND t5.dt = t2.dt
WHERE t1.dt = '{dt}'
    AND t1.tel_result_type = 1
    AND t1.istatus = 1 
    AND t1.talking_time_len >= 15*1000
    AND t2.istatus = 1
    AND get_json_object(t2.voice_info,'$.dialog') != '[]'
    AND t4.area_name LIKE '%电销%'
""")


In [49]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType
# ---------- 2. 加工列 ----------
df_result = sdf_raw.withColumn("processed_sentences", process_sentence_udf(F.col("voice_info")))
df_result.createOrReplaceTempView("tmp")


In [51]:
spark.table('tmp').filter("customer_id='26004183'").show(1, truncate=False)

+-----------+------------------------------------+---------------------------+--------+-------+---------+------------------+-------------------+-------------------+----------------+------------------------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
# 4. 建表（仅首次）
spark.sql("""
CREATE TABLE IF NOT EXISTS mart_caterb2b_mall_risk_management.telemarketing_call_info (
    customer_id            STRING COMMENT '客户ID',
    customer_name          STRING COMMENT '客户名称',
    contact_id             STRING COMMENT '通话id',
    bd_id                  STRING COMMENT '拨打电话的bdid',
    bd_name                STRING COMMENT '拨打电话的bd名称',
    area_name              STRING COMMENT '拨打电话的bd归属区域',
    internet_call_type     STRING COMMENT '网络电话类型 1:木星拨打电话 2:AXB拨打电话',
    talking_start_time     STRING COMMENT '通话开始时间',
    end_time               STRING COMMENT '通话结束时间',
    talking_time_len       STRING COMMENT '通话时长',
    bd_tel_num             STRING COMMENT '脱敏后bd电话',
    customer_tel_num       STRING COMMENT '脱敏后客户电话',
    record_url             STRING COMMENT '通话录音url',
    voice_info             STRING COMMENT '原始转译文本',
    processed_sentences    STRING COMMENT '通话文本内容'
)
COMMENT '电销通话录音转译结果'
PARTITIONED BY (dt STRING COMMENT '日期分区字段，格式yyyymmdd')
STORED AS ORC
""")

# 5. 覆盖写入当天分区
spark.sql(f"""
    INSERT OVERWRITE TABLE mart_caterb2b_mall_risk_management.telemarketing_call_info
    PARTITION (dt='{dt}')
    SELECT customer_id,
           customer_name,
           contact_id,
           bd_id,
           bd_name,
           area_name,
           internet_call_type,
           talking_start_time,
           end_time,
           talking_time_len,
           bd_tel_num,
           customer_tel_num,
           record_url,
           voice_info,
           processed_sentences
    FROM tmp
""")


Moved: 'viewfs://hadoop-meituan/nn27/warehouse/mart_caterb2b_mall_risk_management.db/telemarketing_call_info/dt=20251014' to trash at: hdfs://dfsrouter/user/hadoop-caterb2b-mall-risk-management/.Trash/Current


DataFrame[]